In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0



Databricks


# Build a Customer Support Router Agentic RAG System

In this project, we will leverage the power of AI Agents and RAG Systems to build an intelligent Router Agentic RAG System to handle customer support queries using a custom knowledgebase.

![](https://i.imgur.com/bLCdxCI.png)

### Intelligent Router Agentic RAG System

This project focuses on building an **Intelligent Router Agentic RAG System** that combines intelligent query analysis, sentiment detection, and dynamic routing with Retrieval-Augmented Generation (RAG) to handle diverse user inquiries efficiently. The workflow includes the following components:

1. **Query Categorization and Sentiment Analysis**:
   - The system uses **OpenAI GPT-4o** to analyze the user's query and determine:
     - **Query Category**: Identifies the type of problem, such as billing, technical issues, or general queries.
     - **User Sentiment**: Evaluates the user's sentiment (positive, neutral, or negative) to determine if escalation is needed.

2. **Intelligent Routing**:
   - Based on the **query_category** and **query_sentiment**, the system routes the query to the appropriate handling node:
     - **Escalate to Human**: If the sentiment is negative, the query is escalated to a human for resolution.
     - **Generate Billing Response**: Queries related to billing are routed to generate an appropriate response.
     - **Generate Technical Response**: Technical queries are routed for a specialized technical response.
     - **Generate General Response**: General queries are handled with context-aware responses.

3. **Knowledge Base Integration (RAG)**:
   - The system integrates with a **Knowledge Base (Vector Database)** to augment responses with relevant and accurate information.
   - Retrieval-Augmented Generation (RAG) ensures that responses are grounded in the latest and most reliable data.

4. **Escalation Mechanism**:
   - Negative sentiment triggers an **escalation to a human**, ensuring the user receives empathetic and personalized support for critical issues.


## Install OpenAI, LangGraph and LangChain dependencies

In [0]:
!pip install -q langchain==0.3.14
!pip install -q langchain-openai==0.3.0
!pip install -q langchain-community==0.3.14
!pip install -q langgraph==0.2.64
!pip install -U -qqqq databricks-langchain uv databricks-agents mlflow-skinny[databricks]
!pip install -q databricks-vectorsearch

In [0]:
dbutils.library.restartPython()

## Enter Open AI API Key

In [0]:
# from getpass import getpass

# OPENAI_KEY = getpass('Enter Open AI API Key: ')

## Setup Environment Variables

In [0]:


# os.environ['OPENAI_API_KEY'] = OPENAI_KEY

In [0]:
%%writefile agent.py
##########################################################
# Initial Imports
##########################################################
import os
from databricks.vector_search.client import VectorSearchClient
from langchain_core.documents import Document
from typing import List,Dict,TypedDict, Literal
from pydantic import BaseModel
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from databricks_langchain import ChatDatabricks, UCFunctionToolkit, VectorSearchRetrieverTool
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from mlflow.pyfunc import ResponsesAgent
from mlflow.types.responses import (
    ResponsesAgentRequest,
    ResponsesAgentResponse,
    ResponsesAgentStreamEvent
)
from typing import Annotated, Any, Generator, Optional, Sequence, Union
import mlflow

from langchain_core.messages import (
    AIMessage,
    AIMessageChunk,
    BaseMessage,
    convert_to_openai_messages,
    HumanMessage,
    SystemMessage,
    RemoveMessage,
    trim_messages,
)
from uuid import uuid4  # ⬅️ ADD THIS LINE
import json  # ⬅️ ADD THIS TOO (if not already present)

###########################################################
# Initial Variables Declaration
###########################################################
endpoint_name="router_agent_endpoint"
source_table_name="agentic_ai.langgraph.router_agent_chunks"
index_name="agentic_ai.langgraph.router_agent_index"

###########################################################
# Instantiate the VectorSearchClient to interact with Databricks Vector Search endpoints.
###########################################################
client = VectorSearchClient()
index = client.get_index(endpoint_name=endpoint_name, index_name=index_name)

################################################################################
# Define your LLM endpoint
################################################################################
# llm = ChatOpenAI(model="gpt-4o", temperature=0)
# LLM_ENDPOINT_NAME = "databricks-gpt-oss-120b"
LLM_ENDPOINT_NAME = "databricks-claude-3-7-sonnet"
llm = ChatDatabricks(endpoint=LLM_ENDPOINT_NAME)

######################################################################################################################
# Define the Customer Inquiry State
# We create a CustomerSupportState typed dictionary to keep track of each interaction:

# customer_query: The text of the customer's question
# query_category: Technical, Billing, or General (used for routing)
# query_sentiment: Positive, Neutral, or Negative (used for routing)
# final_response: The system's response to the customer
######################################################################################################################
class CustomerSupportState(TypedDict):
    """
    customer_query: the original query from the customer.
    query_category: the topic of the query (e.g., Technical, Billing).
    query_sentiment: the emotional tone (e.g., Positive, Negative).
    final_response: the system-generated response.
    """
    customer_query: str
    query_category: str
    query_sentiment: str
    messages: str

class QueryCategory(BaseModel):
    categorized_topic: Literal['Technical', 'Billing', 'General']

class QuerySentiment(BaseModel):
    sentiment: Literal['Positive', 'Neutral', 'Negative']

######################################################################################################################
# Create Node Functions
# Each function below represents a stage in processing a customer inquiry:

# categorize_inquiry: Classifies the query into Technical, Billing, or General.
# analyze_inquiry_sentiment: Determines if the sentiment is Positive, Neutral, or Negative.
# generate_technical_response: Produces a response for technical issues.
# generate_billing_response: Produces a response for billing questions.
# generate_general_response: Produces a response for general queries.
# escalate_to_human_agent: Escalates the query to a human if sentiment is negative.
# determine_route: Routes the inquiry to the appropriate response node based on category and sentiment.
######################################################################################################################
def categorize_inquiry(support_state: CustomerSupportState) -> CustomerSupportState:
    """
    Classify the customer query into Technical, Billing, or General.
    """

    query = support_state["customer_query"]
    ROUTE_CATEGORY_PROMPT = """Act as a customer support agent trying to best categorize the customer query.
                               You are an agent for an AI products and hardware company.

                               Please read the customer query below and
                               determine the best category from the following list:

                               'Technical', 'Billing', or 'General'.

                               Remember:
                                - Technical queries will focus more on technical aspects like AI models, hardware, software related queries etc.
                                - General queries will focus more on general aspects like contacting support, finding things, policies etc.
                                - Billing queries will focus more on payment and purchase related aspects

                                Return just the category name (from one of the above)

                                Query:
                                {customer_query}
                            """
    prompt = ROUTE_CATEGORY_PROMPT.format(customer_query=query)
    route_category = llm.with_structured_output(QueryCategory).invoke(prompt)

    return {
        "query_category": route_category.categorized_topic
    }

def analyze_inquiry_sentiment(support_state: CustomerSupportState) -> CustomerSupportState:
    """
    Analyze the sentiment of the customer query as Positive, Neutral, or Negative.
    """

    query = support_state["customer_query"]
    SENTIMENT_CATEGORY_PROMPT = """Act as a customer support agent trying to best categorize the customer query's sentiment.
                                   You are an agent for an AI products and hardware company.

                                   Please read the customer query below,
                                   analyze its sentiment which should be one from the following list:

                                   'Positive', 'Neutral', or 'Negative'.

                                   Return just the sentiment (from one of the above)

                                   Query:
                                   {customer_query}
                                """
    prompt = SENTIMENT_CATEGORY_PROMPT.format(customer_query=query)
    sentiment_category = llm.with_structured_output(QuerySentiment).invoke(prompt)

    return {
        "query_sentiment": sentiment_category.sentiment
    }

def generate_technical_response(support_state: CustomerSupportState) -> CustomerSupportState:
    """
    Provide a technical support response by combining knowledge from the vector store and LLM.
    """
    # Retrieve category and ensure it is lowercase for metadata filtering

    categorized_topic = support_state["query_category"]
    query = support_state["customer_query"]

    # Use metadata filter for 'technical' queries
    if categorized_topic.lower() == "technical":
        # Perform retrieval from VectorDB
        results = index.similarity_search(
                query_text=query,
                columns=["content","category"],  # Ensure only columns present in the index are listed
                num_results=3,
                filters={"category": ["technical"]},
                query_type="hybrid"
            )
        relevant_docs = convert_vector_search_to_documents(results)
        retrieved_content = "\n\n".join(doc.page_content for doc in relevant_docs)

        # Combine retrieved information into the prompt
        prompt = ChatPromptTemplate.from_template(
            """
            Craft a clear and detailed technical support response for the following customer query.
            Use the provided knowledge base information to enrich your response.
            In case there is no knowledge base information or you do not know the answer just say:

            Apologies I was not able to answer your question, please reach out to +1-xxx-xxxx

            Customer Query:
            {customer_query}

            Relevant Knowledge Base Information:
            {retrieved_content}
            """
        )

        # Generate the final response using the LLM
        chain = prompt | llm
        tech_reply = chain.invoke({
            "customer_query": query,
            "retrieved_content": retrieved_content
        }).content
    else:
        # For non-technical queries, provide a default response or a general handling
        tech_reply = "Apologies I was not able to answer your question, please reach out to +1-xxx-xxxx"

    # Update and return the modified support state
    return {
        "messages": tech_reply
    }
    
def generate_billing_response(support_state: CustomerSupportState) -> CustomerSupportState:
    """
    Provide a billing support response by combining knowledge from the vector store and LLM.
    """
    # Retrieve category and ensure it is lowercase for metadata filtering
    categorized_topic = support_state["query_category"]
    query = support_state["customer_query"]

    # Use metadata filter for 'billing' queries
    if categorized_topic.lower() == "billing":
        # Perform retrieval from VectorDB
        results = index.similarity_search(
                query_text=query,
                columns=["content","category"],  # Ensure only columns present in the index are listed
                num_results=3,
                filters={"category": ["billing"]},
                query_type="hybrid"
            )
        relevant_docs = convert_vector_search_to_documents(results)
        retrieved_content = "\n\n".join(doc.page_content for doc in relevant_docs)

        # Combine retrieved information into the prompt
        prompt = ChatPromptTemplate.from_template(
            """
            Craft a clear and detailed billing support response for the following customer query.
            Use the provided knowledge base information to enrich your response.
            In case there is no knowledge base information or you do not know the answer just say:

            Apologies I was not able to answer your question, please reach out to +1-xxx-xxxx

            Customer Query:
            {customer_query}

            Relevant Knowledge Base Information:
            {retrieved_content}
            """
        )

        # Generate the final response using the LLM
        chain = prompt | llm
        billing_reply = chain.invoke({
            "customer_query": query,
            "retrieved_content": retrieved_content
        }).content
    else:
        # For non-billing queries, provide a default response or a general handling
        billing_reply = "Apologies I was not able to answer your question, please reach out to +1-xxx-xxxx"

    # Update and return the modified support state
    return {
        "messages": billing_reply
    }

def generate_general_response(support_state: CustomerSupportState) -> CustomerSupportState:
    """
    Provide a general support response by combining knowledge from the vector store and LLM.
    """
    # Retrieve category and ensure it is lowercase for metadata filtering
    categorized_topic = support_state["query_category"]
    query = support_state["customer_query"]

    # Use metadata filter for 'general' queries
    if categorized_topic.lower() == "general":
        # Perform retrieval from VectorDB
        results = index.similarity_search(
                query_text=query,
                columns=["content","category"],  # Ensure only columns present in the index are listed
                num_results=3,
                filters={"category": ["general"]},
                query_type="hybrid"
            )
        relevant_docs = convert_vector_search_to_documents(results)
        retrieved_content = "\n\n".join(doc.page_content for doc in relevant_docs)
        print(retrieved_content)

        # Combine retrieved information into the prompt
        prompt = ChatPromptTemplate.from_template(
            """
            Craft a clear and detailed general support response for the following customer query.
            Use the provided knowledge base information to enrich your response.
            In case there is no knowledge base information or you do not know the answer just say:

            Apologies I was not able to answer your question, please reach out to +1-xxx-xxxx

            Customer Query:
            {customer_query}

            Relevant Knowledge Base Information:
            {retrieved_content}
            """
        )

        # Generate the final response using the LLM
        chain = prompt | llm
        general_reply = chain.invoke({
            "customer_query": query,
            "retrieved_content": retrieved_content
        }).content
    else:
        # For non-general queries, provide a default response or a general handling
        general_reply = "Apologies I was not able to answer your question, please reach out to +1-xxx-xxxx"

    # Update and return the modified support state
    return {"messages": general_reply}

def escalate_to_human_agent(support_state: CustomerSupportState) -> CustomerSupportState:
    """
    Escalate the query to a human agent if sentiment is negative.
    """

    return {
        "messages": "Apologies, we are really sorry! Someone from our team will be reaching out to your shortly!"
    }

def determine_route(support_state: CustomerSupportState) -> str:
    """
    Route the inquiry based on sentiment and category.
    """
    if support_state["query_sentiment"] == "Negative":
        return "escalate_to_human_agent"
    elif support_state["query_category"] == "Technical":
        return "generate_technical_response"
    elif support_state["query_category"] == "Billing":
        return "generate_billing_response"
    else:
        return "generate_general_response"
    
###########################################################
# Convert the vector search result to LangChain Document
###########################################################
def convert_vector_search_to_documents(results) -> List[Document]:
  column_names = []
  for column in results["manifest"]["columns"]:
      column_names.append(column)

  langchain_docs = []
  for item in results["result"]["data_array"]:
      metadata = {}
      score = item[-1]
      # print(score)
      i = 1
      for field in item[1:-1]:
          # print(field + "--")
          metadata[column_names[i]["name"]] = field
          i = i + 1
      doc = Document(page_content=item[0], metadata=metadata)  # , 9)
      langchain_docs.append(doc)
  return langchain_docs

###############################################################################
# Create Tool-Calling Agent with Language Model and Tools
###############################################################################
def create_router_agent():
    """
    Create a chatbot agent with the given language model and tools.
    """
    # Create the graph with our typed state
    customer_support_graph = StateGraph(CustomerSupportState)

    # Add nodes for each function
    customer_support_graph.add_node("categorize_inquiry", categorize_inquiry)
    customer_support_graph.add_node("analyze_inquiry_sentiment", analyze_inquiry_sentiment)
    customer_support_graph.add_node("generate_technical_response", generate_technical_response)
    customer_support_graph.add_node("generate_billing_response", generate_billing_response)
    customer_support_graph.add_node("generate_general_response", generate_general_response)
    customer_support_graph.add_node("escalate_to_human_agent", escalate_to_human_agent)

    # Add edges to represent the processing flow
    customer_support_graph.add_edge("categorize_inquiry", "analyze_inquiry_sentiment")
    customer_support_graph.add_conditional_edges(
        "analyze_inquiry_sentiment",
        determine_route,
        [
            "generate_technical_response",
            "generate_billing_response",
            "generate_general_response",
            "escalate_to_human_agent"
        ]
    )

    # All terminal nodes lead to the END
    customer_support_graph.add_edge("generate_technical_response", END)
    customer_support_graph.add_edge("generate_billing_response", END)
    customer_support_graph.add_edge("generate_general_response", END)
    customer_support_graph.add_edge("escalate_to_human_agent", END)

    # Set the entry point for the workflow
    customer_support_graph.set_entry_point("categorize_inquiry")

    # Compile the graph into a runnable agent
    memory = MemorySaver()
    compiled_support_agent = customer_support_graph.compile()
    return compiled_support_agent

###############################################################################
# Convert Responses to ChatCompletion Messages
###############################################################################
class LangGraphResponsesAgent(ResponsesAgent):
    def __init__(self, agent):
        self.agent = agent

    def _responses_to_cc(self, message: dict[str, Any]) -> list[dict[str, Any]]:
        """Convert from a Responses API output item to ChatCompletion messages."""
        msg_type = message.get("type")
        print(f"Message type: {msg_type}")
        if msg_type == "function_call":
            return [
                {
                    "role": "assistant",
                    "content": "tool call",
                    "tool_calls": [
                        {
                            "id": message["call_id"],
                            "type": "function",
                            "function": {
                                "arguments": message["arguments"],
                                "name": message["name"],
                            },
                        }
                    ],
                }
            ]
        elif msg_type == "message" and isinstance(message["content"], list):
            return [
                {"role": message["role"], "content": content["text"]}
                for content in message["content"]
            ]
        elif msg_type == "reasoning":
            return [{"role": "assistant", "content": json.dumps(message["summary"])}]
        elif msg_type == "function_call_output":
            return [
                {
                    "role": "tool",
                    "content": message["output"],
                    "tool_call_id": message["call_id"],
                }
            ]
        compatible_keys = ["role", "content", "name", "tool_calls", "tool_call_id"]
        filtered = {k: v for k, v in message.items() if k in compatible_keys}
        return [filtered] if filtered else []


    def _langchain_to_responses(self, messages: list[dict[str, Any]]) -> list[dict[str, Any]]:
        "Convert from ChatCompletion dict to Responses output item dictionaries"
        for message in messages:
            message = message.model_dump()
            role = message["type"]
            if role == "ai":
                if tool_calls := message.get("tool_calls"):
                    return [
                        self.create_function_call_item(
                            id=message.get("id") or str(uuid4()),
                            call_id=tool_call["id"],
                            name=tool_call["name"],
                            arguments=json.dumps(tool_call["args"]),
                        )
                        for tool_call in tool_calls
                    ]
                else:
                    return [
                        self.create_text_output_item(
                            text=message["content"],
                            id=message.get("id") or str(uuid4()),
                        )
                    ]
            elif role == "tool":
                return [
                    self.create_function_call_output_item(
                        call_id=message["tool_call_id"],
                        output=message["content"],
                    )
                ]
            elif role == "user":
                return [message]
            else:
                return [message]

    def predict(self, request: ResponsesAgentRequest) -> ResponsesAgentResponse:
        outputs = [
            event.item
            for event in self.predict_stream(request)
            if event.type == "response.output_item.done"
        ]
        return ResponsesAgentResponse(output=outputs, custom_outputs=request.custom_inputs)

    def predict_stream(
        self,
        request: ResponsesAgentRequest,
    ) -> Generator[ResponsesAgentStreamEvent, None, None]:
        """Handle streaming predictions"""
        try:
            # print(f"🔄 Streaming request received")
            # print(f"Type of request is {type(request)}")
            
            # Convert request to chat completion format
            cc_msgs = []
            for msg in request.input:
                cc_msgs.extend(self._responses_to_cc(msg.model_dump()))
            
            # print(f"chat completion msg is {cc_msgs}")
            # print(f"Type of chat_completion msg is {type(cc_msgs)}")
            
            # Extract user message
            msg = cc_msgs[0]["content"]
            # print(f"🔍 Processing query: {msg}")
            
            # Stream through the agent execution
            final_state = None
            for event in self.agent.stream({"customer_query": msg}, stream_mode=["values"]):
                # Event is a tuple: ('values', state_dict)
                if isinstance(event, tuple) and event[0] == "values":
                    final_state = event[1]  # Extract the actual state dictionary
                    print(f"📊 State update: {list(final_state.keys())}")
            
            # print(f"🎯 Final state: {final_state}")
            
            # After streaming completes, yield the final response
            if final_state and "messages" in final_state and final_state["messages"]:
                response_text = final_state["messages"]
                # print(f"✅ Yielding response: {response_text[:100]}...")
                
                yield ResponsesAgentStreamEvent(
                    type="response.output_item.done",
                    item=self.create_text_output_item(
                        text=response_text,
                        id=str(uuid4())
                    )
                )
            else:
                # print(f"⚠️ No valid response in final state")
                yield ResponsesAgentStreamEvent(
                    type="response.output_item.done",
                    item=self.create_text_output_item(
                        text="No response generated",
                        id=str(uuid4())
                    )
                )
                
        except Exception as e:
            # print(f"❌ Error in predict_stream: {str(e)}")
            import traceback
            traceback.print_exc()
            
            yield ResponsesAgentStreamEvent(
                type="response.output_item.done",
                item=self.create_text_output_item(
                    text=f"Error processing request: {str(e)}",
                    id=str(uuid4())
                )
            )



# Create the agent object, and specify it as the agent object to use when
# loading the agent back for inference via mlflow.models.set_model()
mlflow.langchain.autolog()
agent = create_router_agent()
AGENT = LangGraphResponsesAgent(agent)
mlflow.models.set_model(AGENT)

In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

In [0]:
from agent import AGENT,agent

DEBUG predict_stream

In [0]:
from IPython.display import display, Image,Markdown

Image(agent.get_graph().draw_mermaid_png())

In [0]:
def call_support_agent(agent, prompt, user_session_id, verbose=False):
    events = agent.stream(
        {"customer_query": prompt}, # initial state of the agent
        {"configurable": {"thread_id": user_session_id}},
        stream_mode="values",
    )

    print('Running Agent. Please wait...')
    for event in events:
        print(event)

        # if verbose:
        #         print(event)

    display(Markdown(event['messages']))

## Testing the Customer Support Workflow

Let's test the workflow with some sample queries to verify categorization, sentiment analysis, and response generation.

In [0]:
# from IPython.display import display, Markdown
# result = AGENT.predict({"input": [{"role": "user", "content": "Can you tell me about your shipping policy?"}]})
# display(Markdown(f"{result.output[0].content[0]['text']}"))

In [0]:
# result = AGENT.predict({"input": [{"role": "user", "content": "Can you tell me about your shipping policy?"}]})
# print(result.model_dump(exclude_none=True))

In [0]:
# uid = 'jim001'
# query = "do you support pre-trained models?"
# call_support_agent(agent=agent,
#                    prompt=query,
#                    user_session_id=uid,
#                    verbose=True)

In [0]:
# uid = 'jim002'
# query = "how do I get my invoice?"
# call_support_agent(agent=agent,
#                    prompt=query,
#                    user_session_id=uid,
#                    verbose=True)

In [0]:
import os
from databricks.vector_search.client import VectorSearchClient
from langchain_core.documents import Document
from typing import List,Dict,TypedDict, Literal
from pydantic import BaseModel
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from databricks_langchain import ChatDatabricks, UCFunctionToolkit, VectorSearchRetrieverTool
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from mlflow.pyfunc import ResponsesAgent
from mlflow.types.responses import (
    ResponsesAgentRequest,
    ResponsesAgentResponse,
    ResponsesAgentStreamEvent,
)
from typing import Annotated, Any, Generator, Optional, Sequence, Union
import mlflow
from pkg_resources import get_distribution

In [0]:
from mlflow.models.resources import (
    DatabricksServingEndpoint,
    DatabricksVectorSearchIndex,
)

catalog = "agentic_ai"
schema = "langgraph"
model_name = "customer_support_agent"
UC_MODEL_NAME = f"{catalog}.{schema}.{model_name}"

input = {"input": [{"role": "user", "content": "Can you tell me about your shipping policy?"}]}

with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        name="customer_support_assistant",
        python_model="/Workspace/Users/sourav.banerjee@databricks.com/AgenticAI_Demystified/LANGGRAPH/4. Building Advanced AI Agents with LangGraph/Router Agent/Databricks Compatible/agent.py", 
        pip_requirements=[
            "langchain==0.3.14",
            "langchain-openai==0.3.0",
            "langchain-community==0.3.14",
            "langgraph==0.2.64",
            "databricks-langchain",
            "databricks-agents",
            "mlflow-skinny[databricks]",
            "databricks-vectorsearch",
            "mlflow"
        ],
        resources=[
      DatabricksServingEndpoint(endpoint_name="databricks-claude-3-7-sonnet"),
      DatabricksVectorSearchIndex(index_name="agentic_ai.langgraph.router_agent_index"),
            ],
        registered_model_name=UC_MODEL_NAME,
        input_example= input
            )
# The error was due to incorrect usage of the 'python_model' argument and 'name' instead of 'artifact_path'.

In [0]:
logged_agent_info.run_id 

In [0]:
mlflow.models.predict(
    model_uri=f"runs:/{logged_agent_info.run_id}/customer_support_assistant",
    input_data={"input": [{"role": "user", "content": "Can you tell me about your shipping policy?"}]},
    env_manager="uv",
)

In [0]:
mlflow.set_registry_uri("databricks-uc")

# TODO: define the catalog, schema, and model name for your UC model
catalog = "agents"
schema = "main"
model_name = "langgraph-responses-agent"
UC_MODEL_NAME = UC_MODEL_NAME

# register the model to UC
uc_registered_model_info = mlflow.register_model(model_uri=logged_agent_info.model_uri, name=UC_MODEL_NAME)

In [0]:
token = ""

In [0]:
import os
import requests
import numpy as np
import pandas as pd
import json

def create_tf_serving_json(data):
    return {'inputs': {name: data[name].tolist() for name in data.keys()} if isinstance(data, dict) else data.tolist()}

def score_model(dataset):
    url = 'https://fe-vm-agentic-ai.cloud.databricks.com/serving-endpoints/customer_support_agent/invocations'
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }
    response = requests.post(
        url,
        headers=headers,
        json=dataset
    )
    if response.status_code != 200:
        raise Exception(
            f'Request failed with status {response.status_code}, {response.text}'
        )
    return response.json()

In [0]:
import requests

url = "https://fe-vm-agentic-ai.cloud.databricks.com/serving-endpoints/customer_support_agent/invocations"
headers = {"Authorization": f"Bearer {token}", "Content-Type": "application/json"}

payload = {
    "input": [
        {
            "role": "user",
            "content": "do you support pre-trained models?"
        }
    ]
}

response1 = requests.post(url, json=payload, headers=headers)
response1.text
